In [1]:
from pyomo.environ import *

model = ConcreteModel()

# Sets
model.T = RangeSet(0, 9)

# Parameters
model.C_ch = Param(within=NonNegativeReals, default=10)
model.C_m = Param(domain=NonNegativeReals, default=15)
model.Prod_ch = Param(domain=NonNegativeReals, default=2)
model.Prod_m = Param(domain=NonNegativeReals, default=3)
model.CAP = Param(domain=NonNegativeIntegers, default=100)
model.O_min = Param(domain=NonNegativeIntegers, default=10)

# Demands
D_l = [20.0, 30.0, 20.0, 40.0, 20.0, 25.0, 10.0, 15.0, 12.0, 20.0]
D_c = [30.0, 35.0, 23.0, 31.0, 40.0, 25.0, 20.0, 10.0, 14.0, 20.0]
model.D_l = Param(model.T, domain=NonNegativeReals, initialize=D_l)
model.D_c = Param(model.T, domain=NonNegativeReals, initialize=D_c)

# Variables
model.x = Var(model.T, domain=NonNegativeIntegers, doc='Churras')
model.y = Var(model.T, domain=NonNegativeIntegers, doc='Merinas')

# Constraints
@model.Constraint(model.T, doc='Minimum size')
def minimum_size(m, t):
	if t == 0:
		o_init = 0
	else:
		o_init = m.x[t-1] + m.y[t-1]
	return o_init + m.x[t] + m.y[t] >= m.O_min

@model.Constraint(model.T, doc='Max size')
def max_size(m, t):
	if t == 0:
		o_init = 0
	else:
		o_init = m.x[t-1] + m.y[t-1]
	return o_init + m.x[t] + m.y[t] <= m.CAP

@model.Constraint(model.T, doc='Proportion of churras and merinas')
def proportion(m, t):
	return m.y[t] >= 2*m.x[t]

@model.Constraint(model.T, doc='Wool demand')
def wool_demand(m, t):
	return m.Prod_ch * m.x[t] >= m.D_l[t]

@model.Constraint(model.T, doc='Meat demand')
def meat_demand(m, t):
	return m.Prod_m * m.y[t] >= m.D_c[t]

@model.Objective(sense=minimize)
def objective(m):
	return sum([m.C_ch * m.x[t] + m.C_m * m.y[t] for t in m.T])

In [7]:
model.pprint()

1 RangeSet Declarations
    T : Dimen=1, Size=10, Bounds=(0, 9)
        Key  : Finite : Members
        None :   True :   [0:9]

8 Param Declarations
    CAP : Size=1, Index=None, Domain=NonNegativeIntegers, Default=100, Mutable=False
        Key : Value
    C_ch : Size=1, Index=None, Domain=NonNegativeReals, Default=10, Mutable=False
        Key : Value
    C_m : Size=1, Index=None, Domain=NonNegativeReals, Default=15, Mutable=False
        Key : Value
    D_c : Size=10, Index=T, Domain=NonNegativeReals, Default=None, Mutable=False
        Key : Value
          0 :  30.0
          1 :  35.0
          2 :  23.0
          3 :  31.0
          4 :  40.0
          5 :  25.0
          6 :  20.0
          7 :  10.0
          8 :  14.0
          9 :  20.0
    D_l : Size=10, Index=T, Domain=NonNegativeReals, Default=None, Mutable=False
        Key : Value
          0 :  20.0
          1 :  30.0
          2 :  20.0
          3 :  40.0
          4 :  20.0
          5 :  25.0
          6 :  10.0


In [10]:
# Resolution
solver = SolverFactory("glpk")
solver.solve(model)

# Solution
print("Optimal solution:")
print("X:", [value(model.x[t]) for t in model.T])
print("Y:", [value(model.y[t]) for t in model.T])
print("Total cost:", value(model.objective))

Solución óptima:
X: [10.0, 15.0, 10.0, 20.0, 10.0, 13.0, 5.0, 8.0, 6.0, 10.0]
Y: [20.0, 30.0, 20.0, 40.0, 20.0, 26.0, 10.0, 16.0, 12.0, 20.0]
Coste total: 4280.0
